In [13]:
import pandas as pd

# cargar datos en bruto y eliminar columnas inutiles
original_data = pd.read_csv('../data/_Process_monitoring__202303251137.csv', sep=',')
original_data.drop(columns=['velocity'], inplace=True)

# mostrar info y datos
print(original_data.info())
original_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1685070 entries, 0 to 1685069
Data columns (total 8 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   id          1685070 non-null  int64  
 1   date        1685070 non-null  object 
 2   time        1685070 non-null  object 
 3   lap         1685070 non-null  int64  
 4   latitude    1685070 non-null  float64
 5   longitude   1685070 non-null  float64
 6   navigation  1685070 non-null  int64  
 7   vehicle_id  1685070 non-null  object 
dtypes: float64(2), int64(3), object(3)
memory usage: 102.8+ MB
None


,id,date,time,lap,latitude,longitude,navigation,vehicle_id
0,13173398,2023-03-01,05:10:11,1,-13.549597,-71.877510,285,SJ39
1,13177467,2023-03-01,06:04:51,1,-13.524529,-71.977527,298,SJ34
2,13189674,2023-03-01,07:22:21,1,-13.534183,-71.980709,7,SJ29
3,13192729,2023-03-01,07:39:04,1,-13.551146,-71.986928,202,SJ21
4,13193272,2023-03-01,07:41:52,1,-13.548642,-71.881010,107,SJ34


In [14]:
# copiar el origin data
data = original_data.copy()

# Creacion del las columnas datetime y latitude_longitude
data['datetime'] = data['date'] + ' ' + data['time']
data['latitude_longitude'] = data.apply(lambda row: (row['latitude'], row['longitude']), axis=1)

# eliminar las columnas latitude, longitude, date y time. Luego reordenar
data.drop(columns=['latitude', 'longitude', 'date', 'time'], inplace=True)
data = data[['id', 'datetime', 'latitude_longitude', 'lap', 'navigation', 'vehicle_id']]
data


,id,datetime,latitude_longitude,lap,navigation,vehicle_id
0,13173398,2023-03-01 05:10:11,"(-13.5495973, -71.8775099)",1,285,SJ39
1,13177467,2023-03-01 06:04:51,"(-13.5245292, -71.9775272)",1,298,SJ34
2,13189674,2023-03-01 07:22:21,"(-13.5341828, -71.9807088)",1,7,SJ29
3,13192729,2023-03-01 07:39:04,"(-13.5511464, -71.9869284)",1,202,SJ21
4,13193272,2023-03-01 07:41:52,"(-13.548642, -71.88101)",1,107,SJ34
...,...,...,...,...,...,...
1685065,17241845,2023-03-25 08:50:24,"(-13.5254398, -71.9750786)",2,288,SJ24
1685066,17253753,2023-03-25 10:01:56,"(-13.5511209, -71.9896213)",2,253,SJ16
1685067,17257644,2023-03-25 10:26:10,"(-13.5432069, -71.8977847)",2,126,SJ24
1685068,17266468,2023-03-25 11:14:26,"(-13.5327737, -71.9218652)",1,258,SJ43


### Cleaning dataset

In [47]:
# Verificar y contar si hay puntos duplicados segun datetime y latitude_longitude
twocols_dupli = data.duplicated(subset=['datetime', 'latitude_longitude'])

# Verificar y contar si hay puntos duplicados segun latitude_longitude
onecol_dupli =  data['latitude_longitude'].duplicated()

print(len(twocols_dupli), len(onecol_dupli))
print(twocols_dupli.sum(), onecol_dupli.sum())
print(abs(twocols_dupli.sum() - onecol_dupli.sum()))

tmp = [ not x==y for x,y in zip(twocols_dupli, onecol_dupli)]

data_tmp = data.copy()
data_tmp['h'] = tmp
data_tmp[data_tmp['h'] == True]

1685070 1685070
13116 13707
591


,id,datetime,latitude_longitude,lap,navigation,vehicle_id,h
51965,13382290,2023-03-02 07:52:59,"(-13.5302881, -71.9799186)",1,78,SJ12,True
124433,13466532,2023-03-02 14:47:34,"(-13.5441947, -71.887969)",3,290,SJ40,True
146431,13560399,2023-03-03 07:21:00,"(-13.530429, -71.9313513)",1,290,SJ07,True
166375,13476052,2023-03-02 15:38:11,"(-13.5240449, -71.9831016)",4,10,SJ38,True
169091,13553895,2023-03-03 06:46:12,"(-13.528071, -71.9826652)",1,92,SJ27,True
...,...,...,...,...,...,...,...
1680966,17188596,2023-03-24 20:09:13,"(-13.5462549, -71.8901446)",5,103,SJ27,True
1681443,17241691,2023-03-25 08:49:32,"(-13.5299267, -71.9330024)",2,282,SJ20,True
1682065,17112893,2023-03-24 13:35:46,"(-13.5533389, -71.8615486)",3,277,SJ12,True
1684225,17148135,2023-03-24 16:38:33,"(-13.5505533, -71.9890628)",5,232,SJ34,True


13707

In [ ]:
# diseño del df: creacion data_time y latitude_longitude
data['date_time'] = data['date'] + ' ' + data['time']
data['latitude'] = data.apply(lambda row: (row['latitude'], row['longitude']), axis=1)
data.drop(['longitude'], inplace=True, axis=1)
data = data.rename(columns={'latitude':'latitude_longitude', 'vehicle_id_id':'vehicle_id'}, inplace=False)
data = data.drop_duplicates(subset=['latitude_longitude']) # (65155, 7) (64702, 7)
data = data.sort_values(by='date_time')

# convertir de object a datatime
data['date_time'] = pd.to_datetime(data['date_time'], format='%Y-%m-%d %H:%M:%S')
print(data.shape)
data

In [8]:
stops = pd.read_csv('../data/busstops_ida.csv', sep=',')
stops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   number      84 non-null     int64  
 1   id          84 non-null     int64  
 2   name        84 non-null     object 
 3   navigation  84 non-null     int64  
 4   latitude    84 non-null     float64
 5   longitude   84 non-null     float64
 6   radio       84 non-null     int64  
dtypes: float64(2), int64(4), object(1)
memory usage: 4.7+ KB
